In [1]:
#Ethan Vaz Falcao 
import os 
import pandas as pd  

Data sourced from kaggle.com
https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017?resource=download&select=shootouts.csv

In [2]:
df_fifa_rank = pd.read_csv("/Users/Ethan Vaz Falcao/Downloads/Data/fifa_ranking-2022-10-06.csv", sep=",")
df_fifa_results = pd.read_csv("/Users/Ethan Vaz Falcao/Downloads/Data/results.csv", sep=",")

In [3]:
df_fifa_rank.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31


In [4]:
df_fifa_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [5]:
#Convert the values in the "date" and "rank_data" of the df_fifa_results and df_fifa_rank, to represent date times. 
df_fifa_results["date"] = pd.to_datetime(df_fifa_results["date"])
df_fifa_rank["rank_date"] = pd.to_datetime(df_fifa_rank["rank_date"])
df_fifa_rank = df_fifa_rank.set_index(['rank_date']).groupby(['country_full'], group_keys=False).resample('D').first().fillna(method='ffill').reset_index()

In [6]:
# Merging the df_fifa_rank and df_fifa_results together 
df = df_fifa_results.merge(df_fifa_rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "home_team"], right_on=["rank_date", "country_full"])
df = df.merge(df_fifa_rank[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "away_team"], right_on=["rank_date", "country_full"], suffixes=("_home", "_away"))

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21483 entries, 0 to 21482
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  21483 non-null  datetime64[ns]
 1   home_team             21483 non-null  object        
 2   away_team             21483 non-null  object        
 3   home_score            21483 non-null  int64         
 4   away_score            21483 non-null  int64         
 5   tournament            21483 non-null  object        
 6   city                  21483 non-null  object        
 7   country               21483 non-null  object        
 8   neutral               21483 non-null  bool          
 9   country_full_home     21483 non-null  object        
 10  total_points_home     21483 non-null  float64       
 11  previous_points_home  21483 non-null  float64       
 12  rank_home             21483 non-null  float64       
 13  rank_change_home

In [8]:
# Dropping the repeated columns from merge 
df= df.drop(["rank_date_home","rank_date_away", "country_full_away","country_full_home",
             "rank_change_away","rank_change_home","country_full_away","country_full_home","previous_points_away","previous_points_home","country","neutral"], axis=1)

In [9]:
#renaming column headers and fixing any contry names
df = df.rename(columns = {'home_score': "home_team_score"})
df = df.rename(columns = {'away_score': "away_team_score"})
df = df.rename(columns = {'rank_home': "home_team_fifa_rank"})
df = df.rename(columns = {'rank_away': "away_team_fifa_rank"})
df = df.rename(columns = {'total_points_home': "home_team_total_fifa_points"})
df = df.rename(columns = {'total_points_away': "away_team_total_fifa_points"})
df = df.rename(columns = {'city': "location"})
df["home_team"] = df["home_team"].str.replace("USA", "United States").str.replace("IR Iran", "Iran").str.replace("Israel", "Palestine")

In [21]:
df["date"] = pd.to_datetime(df["date"])
df = df[(df["date"] >= "2008-8-1")].reset_index(drop=True)

In [22]:
df["goal_difference"] = (
    abs(df["home_team_score"] - df["away_team_score"])
)
df["goal_total"] = (
    abs(df["home_team_score"] + df["away_team_score"])
)
df.head()

,date,home_team,away_team,home_team_score,away_team_score,tournament,location,home_team_total_fifa_points,home_team_fifa_rank,away_team_total_fifa_points,away_team_fifa_rank,goal_difference,goal_total
0,2008-08-01,Afghanistan,Turkmenistan,0,5,AFC Challenge Cup,Hyderabad,53.0,181.0,127.0,158.0,5,5
1,2008-08-01,India,Tajikistan,1,1,AFC Challenge Cup,Hyderabad,137.0,153.0,135.0,154.0,0,2
2,2008-08-02,Sri Lanka,Myanmar,1,3,AFC Challenge Cup,Hyderabad,111.0,161.0,100.0,164.0,2,4
3,2008-08-03,Afghanistan,Tajikistan,0,4,AFC Challenge Cup,Hyderabad,53.0,181.0,135.0,154.0,4,4
4,2008-08-03,India,Turkmenistan,2,1,AFC Challenge Cup,Hyderabad,137.0,153.0,127.0,158.0,1,3


In [23]:
df

,date,home_team,away_team,home_team_score,away_team_score,tournament,location,home_team_total_fifa_points,home_team_fifa_rank,away_team_total_fifa_points,away_team_fifa_rank,goal_difference,goal_total
0,2008-08-01,Afghanistan,Turkmenistan,0,5,AFC Challenge Cup,Hyderabad,53.00,181.0,127.00,158.0,5,5
1,2008-08-01,India,Tajikistan,1,1,AFC Challenge Cup,Hyderabad,137.00,153.0,135.00,154.0,0,2
2,2008-08-02,Sri Lanka,Myanmar,1,3,AFC Challenge Cup,Hyderabad,111.00,161.0,100.00,164.0,2,4
3,2008-08-03,Afghanistan,Tajikistan,0,4,AFC Challenge Cup,Hyderabad,53.00,181.0,135.00,154.0,4,4
4,2008-08-03,India,Turkmenistan,2,1,AFC Challenge Cup,Hyderabad,137.00,153.0,127.00,158.0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10943,2022-09-27,Norway,Serbia,0,2,UEFA Nations League,Oslo,1488.57,36.0,1549.53,25.0,2,2
10944,2022-09-27,Sweden,Slovenia,1,1,UEFA Nations League,Stockholm,1563.44,20.0,1372.48,65.0,0,2
10945,2022-09-27,Kosovo,Cyprus,5,1,UEFA Nations League,Pristina,1183.90,106.0,1180.52,108.0,4,6
10946,2022-09-27,Greece,Northern Ireland,3,1,UEFA Nations League,Athens,1441.45,49.0,1399.10,58.0,2,4


# Dataframe "df_Fifa_world_cup" Column Description
#### date: date of the match
##### home_team: the name of the home team
#### away_team: the name of the away team
#### home_team_score: full-time home team score
#### away_team_score: full-time away team score 
#### tournament: the name of the tournament
#### location: the name of the city where the match was played
#### home_team_total_fifa_points: current total points of home team as of the time of the match
#### home_team_fifa_rank: current country rank of home team as of the time of the match
#### away_team_total_fifa_points: current total points of away team as of the time of the match
#### away_team_fifa_rank: country rank of away team as of the time of the match 
#### goal_difference : The absolute value of the difference in goals scored between two countries 
#### goal_total: The totle of goals scored in a match 


In [24]:
df_Fifa_world_cup = df
df_Fifa_world_cup.head()

,date,home_team,away_team,home_team_score,away_team_score,tournament,location,home_team_total_fifa_points,home_team_fifa_rank,away_team_total_fifa_points,away_team_fifa_rank,goal_difference,goal_total
0,2008-08-01,Afghanistan,Turkmenistan,0,5,AFC Challenge Cup,Hyderabad,53.0,181.0,127.0,158.0,5,5
1,2008-08-01,India,Tajikistan,1,1,AFC Challenge Cup,Hyderabad,137.0,153.0,135.0,154.0,0,2
2,2008-08-02,Sri Lanka,Myanmar,1,3,AFC Challenge Cup,Hyderabad,111.0,161.0,100.0,164.0,2,4
3,2008-08-03,Afghanistan,Tajikistan,0,4,AFC Challenge Cup,Hyderabad,53.0,181.0,135.0,154.0,4,4
4,2008-08-03,India,Turkmenistan,2,1,AFC Challenge Cup,Hyderabad,137.0,153.0,127.0,158.0,1,3


In [25]:
df_Fifa_world_cup.to_csv(r'/Users/Ethan Vaz Falcao/Downloads/Data/Fifa_world_cup.csv')